# F05. Generate LULC Map

## F05.00. Initialize Earth Engine

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ee
import geemap
from gee_utils import get_aoi_from_gaul, get_landsat_composite, add_spectral_indices, split_training_validation, sample_composite

In [ ]:
# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

## User Inputs and Parameters

In [44]:
# --- Area of Interest (AOI) ---
# Set the country and province for the AOI using GAUL admin boundaries
COUNTRY = "Indonesia"
PROVINCE = "Sumatera Selatan"

# --- Time Period ---
# Set the analysis year and date range
START_DATE = '2018-01-01'
END_DATE = '2018-12-31'

# --- Landsat Settings ---
# Choose Landsat version: 'LC08' for Landsat 8, 'LC09' for Landsat 9
LANDSAT_VERSION = 'LC08'
CLOUD_COVER = 50  # Maximum cloud cover percentage

# --- Bands to Use ---
# List of bands and indices to use for classification
BANDS = [
    'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7',  # Surface reflectance bands
    'NDVI', 'NBR', 'NDWI', 'EVI2'                          # Spectral indices
]

# --- Reference Data ---
# Path to the ground truth points FeatureCollection (should contain 17 LULC classes)
TRAINING_POINTS_ASSET = 'projects/ee-rg2icraf/assets/Sumsel_GT_Restore'


# --- Visualization ---
# Color palette for each LULC class (order must match class names)
land_cover_palette = [
    '#006400',  # Undisturbed dry-land forest
    '#228B22',  # Logged-over dry-land forest
    '#4169E1',  # Undisturbed mangrove
    '#87CEEB',  # Logged-over mangrove
    '#2E8B57',  # Undisturbed swamp forest
    '#8FBC8F',  # Logged-over swamp forest
    '#9ACD32',  # Agroforestry
    '#32CD32',  # Plantation forest
    '#8B4513',  # Rubber monoculture
    '#FF8C00',  # Oil palm monoculture
    '#DAA520',  # Other monoculture
    '#ADFF2F',  # Grass/savanna
    '#90EE90',  # Shrub
    '#FFFF00',  # Cropland
    '#FF0000',  # Settlement
    '#D2B48C',  # Cleared land
    '#0000FF'   # Waterbody
]

land_cover_names = [
    'Undisturbed dry-land forest',
    'Logged-over dry-land forest',
    'Undisturbed mangrove',
    'Logged-over mangrove',
    'Undisturbed swamp forest',
    'Logged-over swamp forest',
    'Agroforestry',
    'Plantation forest',
    'Rubber monoculture',
    'Oil palm monoculture',
    'Other monoculture',
    'Grass/savanna',
    'Shrub',
    'Cropland',
    'Settlement',
    'Cleared land',
    'Waterbody'
]

## F05.01. Image Acquisition & Composite Generation

### F05.01.A. Load Area of Interest

In [45]:

aoi = get_aoi_from_gaul(country=COUNTRY, province=PROVINCE)
# Visualise the AOI
Map = geemap.Map() 
Map.addLayer(aoi, 
            {'color': 'red', 'fillColor': '00000000'}, 
            'Area of Interest (AOI)',)
# Visualize AOI
Map.centerObject(aoi, 8)
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

### F05.01.B. Generate Composite

In [46]:
landsat_composite = get_landsat_composite(
    aoi=aoi,
    start_date=START_DATE,
    end_date=END_DATE,
    landsat_version=LANDSAT_VERSION,
    cloud_cover=CLOUD_COVER
)

Map = geemap.Map()
Map.centerObject(aoi, 8)

Map.addLayer(landsat_composite, 
            {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3}, 
            'Composite (RGB)')

# Display composite
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## F05.02. Generate and Select Covariates

### F05.02.A. Calculate Spectral indices

In [47]:
# Input bands for generate NDVI and NDWI
composite_with_indices = add_spectral_indices(landsat_composite)

# Select NDVI and NDWI bands from the composite
ndvi = composite_with_indices.select('NDVI')
ndwi = composite_with_indices.select('NDWI')

# Add NDVI layer
ndvi_palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9641']

Map = geemap.Map()
Map.centerObject(aoi, 8)

Map.addLayer(ndvi.clip(aoi), 
            {'min': -1, 'max': 1, 'palette': ndvi_palette}, 
            'NDVI')

# Add NDWI layer
ndwi_palette = ['#8B4513', '#DAA520', '#FFFF00', '#ADFF2F', '#00FF00', '#00FFFF', '#0000FF', '#000080']
Map.addLayer(ndwi.clip(aoi), 
            {'min': -1, 'max': 1, 'palette': ndwi_palette}, 
            'NDWI')
# Display map
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## F05.03. Define LULC Categories & Reference Data Preparation

### F05.03.A. Prepare training and validation data

In [48]:
training_points = ee.FeatureCollection(TRAINING_POINTS_ASSET)
# Check training data size
print('Total training points:', training_points.size().getInfo())

# Split points into training and validation sets
training, validation = split_training_validation(training_points, split=0.7, seed=42)


print('Training points:', training.size().getInfo())
print('Validation points:', validation.size().getInfo())

# Add training and validation points separately
Map = geemap.Map()
Map.centerObject(aoi, 8)

Map.addLayer(training, 
            {'color': 'blue'}, 
            'Training Points')

Map.addLayer(validation, 
            {'color': 'orange'}, 
            'Validation Points')

# Display points data
Map

Total training points: 39003
Training points: 27182
Validation points: 11821


Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## F05.04. Feature Extraction

### F05.04.A. Optimized sampling

In [49]:
# Input selected bands
training_samples = sample_composite(composite_with_indices, training, BANDS)
validation_samples = sample_composite(composite_with_indices, validation, BANDS)

## F05.05. Model Training & Classification

### F05.05.A. Model training using RandomForest

In [50]:
classifier = ee.Classifier.smileRandomForest(
    # Input hyper-parameter
    numberOfTrees=100,  # Reduced from 200
    variablesPerSplit=3,
    minLeafPopulation=2,  # Increased from 1
    bagFraction=0.7,  # Increased from 0.5
    seed=42
)

trained = classifier.train(
    features=training_samples,
    classProperty='kelas',
    inputProperties=BANDS
)

### F05.05.B. Classification

In [51]:
# Classify with tileScale for memory optimization
classified = (composite_with_indices.select(BANDS)
              .classify(trained)
              .set('system:time_start', ee.Date(START_DATE).millis()))

### F05.05.C. Validation

In [52]:
validated = validation_samples.classify(trained)
confusion_matrix = validated.errorMatrix('kelas', 'classification')

print('=== ACCURACY RESULTS ===')
print('Overall Accuracy:', confusion_matrix.accuracy().getInfo())
print('Kappa Coefficient:', confusion_matrix.kappa().getInfo())


=== ACCURACY RESULTS ===
Overall Accuracy: 0.5912836632401665
Kappa Coefficient: 0.5322085224807218


### F05.05.D. Visualisation

In [40]:
# Create interactive map
Map = geemap.Map()
Map.centerObject(aoi, 8)

# Add RGB composite layer
Map.addLayer(
    landsat_composite,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'RGB Composite'
)

# Add classified land cover layer
Map.addLayer(
    classified.clip(aoi),
    {'min': 1, 'max': 17, 'palette': land_cover_palette},
    'Land Cover Classification'
)

# Create legend 
legend_dict = dict(zip(land_cover_names, land_cover_palette))
Map.add_legend(
    title="Land Cover Classes",
    legend_dict=legend_dict,
    draggable=True
)

# Add layer control and display
Map.addLayerControl()
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

### F05.05.E. Export result

In [ ]:
def export_to_drive():
    """Export classification results to Google Drive"""
    
    # Export land cover classification
    year = int(START_DATE[:4])
    task1 = ee.batch.Export.image.toDrive(
        image=classified.clip(aoi),
        description=f'LandCover_17Classes_Optimized_{year}',
        folder='Epsitem_Sumsel',
        region=aoi,
        scale=30,
        maxPixels=1e9,
        crs='EPSG:4326',
        fileFormat='GeoTIFF',
        formatOptions={'cloudOptimized': True}
    )